In [34]:
from google.colab import drive
drive.mount('/content/drive')


In [35]:
dataset_path = '/content/drive/MyDrive/Solanum tuberosum -project/training/PlantVillage'

In [36]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [37]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 50

In [38]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [39]:
class_names = dataset.class_names
class_names

In [40]:
import matplotlib.pyplot as plt

# Set up the plot with a specific figure size
plt.figure(figsize=(10, 10))

# Take one batch of images and labels from the dataset
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)  # Print the shape of the image batch
    print(labels_batch.numpy())  # Print the labels in numpy format

    # Display 12 images from the batch
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)  # Create a 3x4 grid of subplots
        plt.imshow(image_batch[i].numpy().astype("uint8"))  # Display the image
        plt.title(class_names[labels_batch[i]])  # Set the title to the class name
        plt.axis("off")  # Turn off the axis

# Show the plot
plt.show()

In [41]:
train_size = 0.8
len(dataset) * train_size

In [42]:
train_ds = dataset.take(54)  # Take 54 batches for training
len(train_ds)

In [43]:
test_ds = dataset.skip(54)  # Skip the first 54 batches for the test dataset
len(test_ds)

In [44]:
val_size = 0.1
len(dataset) * val_size

In [45]:
val_ds = test_ds.take(6)  # Take the first 6 batches from the test dataset for validation
len(val_ds)

In [46]:
test_ds = test_ds.skip(6)  # Skip the first 6 batches from the test dataset
len(test_ds)

In [47]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [48]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [49]:
len(train_ds)

In [50]:
len(val_ds)

In [51]:
len(test_ds)

In [52]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [53]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch[0].numpy()/255)

In [54]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.Rescaling(1.0/255)
])

In [55]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

In [56]:
BATCH_SIZE

In [57]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

In [58]:
model.summary()

In [59]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [60]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS
)

In [61]:
len(test_ds)

In [62]:
scores = model.evaluate(test_ds)

In [63]:
scores

In [64]:
history.history.keys()

In [65]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [66]:
history.history['accuracy']

In [67]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [71]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    first_image = images_batch[0].numpy().astype("uint8")
    print("First image to predict")
    plt.imshow(first_image)
    print("Actual Label:", class_names[labels_batch[0].numpy()])

    batch_prediction = model.predict(images_batch)
    print("Predicted Label:", class_names[np.argmax(batch_prediction[0])])

In [72]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [73]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())

        actual_class = class_names[labels[i]]
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class},\n Confidence: {confidence}%")
        plt.axis("off")

In [82]:
from google.colab import drive
drive.mount('/content/drive')

In [83]:
import os

# Define the base directory where all model versions will be saved
base_dir = '/content/drive/MyDrive/Solanum tuberosum -project/saved_models'

# Create the directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# List existing model versions and find the highest version
existing_versions = os.listdir(base_dir)
if existing_versions:
    model_version = max([int(v) for v in existing_versions if v.isdigit()]) + 1
else:
    model_version = 1  # Start with version 1 if no versions exist

print(f"Saving model as version: {model_version}")


In [84]:
# Define the path for this version of the model
model_save_path = f'{base_dir}/{model_version}/model.keras'

# Create the directory for the version if it doesn't exist
os.makedirs(f'{base_dir}/{model_version}', exist_ok=True)

# Save the model
model.save(model_save_path)

print(f"Model saved in version: {model_version}")


In [77]:
# List the existing model versions, if any
existing_versions = os.listdir(base_dir)

# If there are existing versions, find the highest version number
if existing_versions:
    model_version = max([int(v) for v in existing_versions if v.isdigit()]) + 1
else:
    model_version = 1  # Start from version 1 if no versions exist

print(f"Saving model version: {model_version}")


In [85]:
# Define the full path for this version of the model
model_save_path = f'{base_dir}/{model_version}/model.keras'  # Save the model in .keras format

# Create the directory for this version if it doesn't exist
os.makedirs(f'{base_dir}/{model_version}', exist_ok=True)

# Save the model
model.save(model_save_path)

print(f"Model saved in version: {model_version} in .keras format")


In [86]:
# Load the model from the .keras file
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/Solanum tuberosum -project/saved_models/1/model.keras')

# Print model summary to verify
loaded_model.summary()


In [87]:
from google.colab import drive
drive.mount('/content/drive')

In [88]:
import os

# Define the path for the api directory
api_dir = '/content/drive/MyDrive/Solanum tuberosum -project/api'

# Create the directory if it doesn't exist
if not os.path.exists(api_dir):
    os.makedirs(api_dir)
    print(f"Directory '{api_dir}' created!")
else:
    print(f"Directory '{api_dir}' already exists.")


In [89]:
# Path to the requirements.txt file
requirements_path = os.path.join(api_dir, 'requirements.txt')

# Write the required dependencies into the requirements.txt file
with open(requirements_path, 'w') as f:
    f.write("tensorflow==2.5.0\n")
    f.write("fastapi\n")
    f.write("uvicorn\n")
    f.write("python-multipart\n")
    f.write("pillow\n")
    f.write("tensorflow-serving-api==2.5.0\n")
    f.write("matplotlib\n")
    f.write("numpy\n")

print(f"requirements.txt created at {requirements_path}")
